<font color='green'>**CUSTOMER SEGMENT**</font> 

WORKFLOW:

1. Lấy các biến từ 2 nhóm biến thông tin tài chính (bao gồm hành vi về tiền gửi thanh toán và tiền gửi tiết kiệm của khách hàng) và thông tin về dư nợ và dư nợ gốc của khách hàng
2. Chạy mô hình Decision tree với 2 tập biến riêng biệt đó để chọn lựa biến quan trọng trong việc phân biệt khách hàng default
3. Sau khi chọn được mỗi biến quan trọng nhất từ từng nhóm biến, tiến hành chạy mô hình Decision tree để tiếp tục chọn ra biến quan trọng từ 2 nhóm biến
4. Trong trường hợp độ quan trọng của 2 biến là tương đương, sẽ tiến hành phân khúc dựa trên cả 2 biến. Trong trường hợp còn lại, biến quan trọng hơn sẽ được lấy ra để phân khúc khách hàng
5. Việc phân khúc khách hàng cuối cùng sẽ được thực hiện với mô hình Decision tree

<font color='red'>**Lưu ý 1: Mỗi khi chạy mô hình phân khúc, các biến sẽ được tạo thêm 1 phiên bản đã biến đổi log (do phân phối của biến không phải phân phối chuẩn) phục vụ cho việc cải thiện khả năng phân biệt (nếu có thể)**</font> 

<font color='red'>**Lưu ý 2: Việc phân khúc sẽ được tính toán thực hiện trên toàn bộ dữ liệu phục vụ cho việc training, testing, out of time sampling để có thể split data xây dựng mô hình**</font> 

### 1. Import lib and data

In [2]:
# Import lib and data
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import optuna
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore", category=DeprecationWarning)

# #warnings.filterwarnings('ignore')
# os.chdir("F:\Data")

### 2. Import data

In [4]:
# Import dữ liệu lớn bằng Polars
file_pl = pl.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')

In [9]:
from duckdb import sql
sql("""
select * from file_pl 
""").columns

['datadate',
 'SOCIF',
 'CBALORG',
 'CBALORG_AVG',
 'CBALORG_MAX',
 'CBALORG_MIN',
 'CBAL',
 'CBAL_AVG',
 'CBAL_MAX',
 'CBAL_MIN',
 'NHANVIENBIDV',
 'PURCOD_MAX',
 'PURCOD_MIN',
 'AFLIMT_SUM',
 'AFLIMT_MAX',
 'AFLIMT_MIN',
 'AFLIMT_AVG',
 'DURATION_AVG',
 'DURATION_MAX',
 'DURATION_MIN',
 'REMAINING_DURATION_AVG',
 'REMAINING_DURATION_MAX',
 'REMAINING_DURATION_MIN',
 'TIME_TO_OP_AVG',
 'TIME_TO_OP_MAX',
 'TIME_TO_OP_MIN',
 'RATE_AVG',
 'RATE_MAX',
 'RATE_MIN',
 'PRINPICAL_PYMT_FRQ_ID_MAX',
 'INT_PYMT_FRQ_ID_MAX',
 'PRINPICAL_PYMT_FRQ_ID_MIN',
 'INT_PYMT_FRQ_ID_MIN',
 'CFORGD',
 'FLAG_SALARY_ACC',
 'TUOI',
 'TRINHDO',
 'TTHONNHAN',
 'SOHUUNHA',
 'LTV',
 'MAX_LTV_MO',
 'MIN_LTV_MO',
 'AVG_LTV_MO',
 'N_G_LTV_3M',
 'N_G_LTV_6M',
 'N_G_LTV_9M',
 'N_G_LTV_12M',
 'N_MIN_LTV',
 'N_MAX_LTV',
 'N_AVG_LTV',
 'INCOME',
 'CBAL_TO_INC_3MON',
 'CBAL_TO_INC_6MON',
 'CBAL_TO_INC_9MON',
 'CBAL_TO_INC_12MON',
 'FLAG_DEPOSIT',
 'N_AVG_CD_3M',
 'N_AVG_CD_6M',
 'N_AVG_CD_9M',
 'N_AVG_CD_12M',
 'N_MAX_CD_CB

In [9]:
from duckdb import sql
sql("""
select * from file_pl 
""").columns

['datadate',
 'SOCIF',
 'CBALORG',
 'CBALORG_AVG',
 'CBALORG_MAX',
 'CBALORG_MIN',
 'CBAL',
 'CBAL_AVG',
 'CBAL_MAX',
 'CBAL_MIN',
 'NHANVIENBIDV',
 'PURCOD_MAX',
 'PURCOD_MIN',
 'AFLIMT_SUM',
 'AFLIMT_MAX',
 'AFLIMT_MIN',
 'AFLIMT_AVG',
 'DURATION_AVG',
 'DURATION_MAX',
 'DURATION_MIN',
 'REMAINING_DURATION_AVG',
 'REMAINING_DURATION_MAX',
 'REMAINING_DURATION_MIN',
 'TIME_TO_OP_AVG',
 'TIME_TO_OP_MAX',
 'TIME_TO_OP_MIN',
 'RATE_AVG',
 'RATE_MAX',
 'RATE_MIN',
 'PRINPICAL_PYMT_FRQ_ID_MAX',
 'INT_PYMT_FRQ_ID_MAX',
 'PRINPICAL_PYMT_FRQ_ID_MIN',
 'INT_PYMT_FRQ_ID_MIN',
 'CFORGD',
 'FLAG_SALARY_ACC',
 'TUOI',
 'TRINHDO',
 'TTHONNHAN',
 'SOHUUNHA',
 'LTV',
 'MAX_LTV_MO',
 'MIN_LTV_MO',
 'AVG_LTV_MO',
 'N_G_LTV_3M',
 'N_G_LTV_6M',
 'N_G_LTV_9M',
 'N_G_LTV_12M',
 'N_MIN_LTV',
 'N_MAX_LTV',
 'N_AVG_LTV',
 'INCOME',
 'CBAL_TO_INC_3MON',
 'CBAL_TO_INC_6MON',
 'CBAL_TO_INC_9MON',
 'CBAL_TO_INC_12MON',
 'FLAG_DEPOSIT',
 'N_AVG_CD_3M',
 'N_AVG_CD_6M',
 'N_AVG_CD_9M',
 'N_AVG_CD_12M',
 'N_MAX_CD_CB

In [7]:
y = file_pl.select('BAD')

### 3. Variables selection by SelectFromModel

In [9]:
X_1 = file_pl.select(['INCOME',
 'N_AVG_CD_3M',
 'N_AVG_CD_6M',
 'N_AVG_CD_9M',
 'N_AVG_CD_12M',
 'N_AVG_DD_3M',
 'N_AVG_DD_6M',
 'N_AVG_DD_9M',
 'N_AVG_DD_12M',
 'N_AVG_DEPOSIT_3M',
 'N_AVG_DEPOSIT_6M',
 'N_AVG_DEPOSIT_9M',
 'N_AVG_DEPOSIT_12M'])

for i in X_1.columns:
    X_1 = X_1.with_columns(
        pl.when(pl.col(i) > 0)
        .then(pl.col(i).log()) 
        .otherwise(np.log(1e-10))  
        .alias(f"{i}_log")
    )

model_1 = DecisionTreeClassifier(
        criterion = 'gini',
        splitter='best',
        random_state=123,
        class_weight='balanced')

X_1_np = X_1.to_numpy()

y_np = y.to_numpy().ravel()

model_1.fit(X_1_np, y_np)

DecisionTreeClassifier(class_weight='balanced', random_state=123)

In [10]:
# Lấy độ quan trọng
importances = model_1.feature_importances_
feature_names = X_1.columns  # Nếu X là DataFrame

# Tạo bảng
feat_imp = pd.Series(importances, index=feature_names).sort_values(ascending=False)

# In ra top feature quan trọng nhất
print("🎯 Feature quan trọng:")
feat_imp

🎯 Feature quan trọng:


N_AVG_DEPOSIT_3M_log     0.120402
N_AVG_DD_3M              0.061866
N_AVG_DEPOSIT_12M        0.061445
N_AVG_DD_12M_log         0.058038
N_AVG_DD_12M             0.053558
N_AVG_DD_3M_log          0.051667
N_AVG_DEPOSIT_12M_log    0.051193
N_AVG_DEPOSIT_3M         0.050949
INCOME_log               0.050612
INCOME                   0.049042
N_AVG_DD_6M              0.048381
N_AVG_DD_6M_log          0.048069
N_AVG_DEPOSIT_9M         0.047444
N_AVG_DEPOSIT_6M         0.047196
N_AVG_DEPOSIT_6M_log     0.047109
N_AVG_DEPOSIT_9M_log     0.046498
N_AVG_DD_9M              0.046472
N_AVG_DD_9M_log          0.046168
N_AVG_CD_12M_log         0.003022
N_AVG_CD_12M             0.002874
N_AVG_CD_3M              0.001984
N_AVG_CD_3M_log          0.001509
N_AVG_CD_9M_log          0.001283
N_AVG_CD_9M              0.001227
N_AVG_CD_6M_log          0.001014
N_AVG_CD_6M              0.000978
dtype: float64

In [25]:
X_2 = file_pl.select(['CBAL',
 'CBALORG',
 'CBALORG_AVG',
 'CBAL_AVG',
 'N_AVG_CBAL_12M',
 'N_AVG_CBAL_9M',
 'N_AVG_CBAL_6M',
 'N_AVG_CBAL_3M'])

for i in X_2.columns:
    X_2 = X_2.with_columns(
        pl.when(pl.col(i) > 0)
        .then(pl.col(i).log()) 
        .otherwise(np.log(1e-10))  
        .alias(f"{i}_log")
    )

model_2 = DecisionTreeClassifier(
        criterion = 'gini',
        splitter='best',
        random_state=123,
        class_weight='balanced')

X_2_np = X_2.to_numpy()

y_np = y.to_numpy().ravel()

model_2.fit(X_2_np, y_np)

DecisionTreeClassifier(class_weight='balanced', random_state=123)

In [26]:
# Lấy độ quan trọng
importances = model_2.feature_importances_
feature_names = X_2.columns  # Nếu X là DataFrame

# Tạo bảng
feat_imp = pd.Series(importances, index=feature_names).sort_values(ascending=False)

# In ra top feature quan trọng nhất
print("🎯 Feature quan trọng:")
feat_imp

🎯 Feature quan trọng:


CBAL                  0.108323
CBAL_AVG              0.098742
CBAL_log              0.089828
CBAL_AVG_log          0.088708
N_AVG_CBAL_3M         0.066741
N_AVG_CBAL_12M        0.062325
N_AVG_CBAL_3M_log     0.059543
N_AVG_CBAL_12M_log    0.059445
N_AVG_CBAL_6M         0.052279
N_AVG_CBAL_6M_log     0.047737
CBALORG_log           0.047122
CBALORG               0.046811
CBALORG_AVG           0.046363
N_AVG_CBAL_9M         0.043910
N_AVG_CBAL_9M_log     0.041222
CBALORG_AVG_log       0.040902
dtype: float64

In [1]:
from duckdb import sql

In [7]:
sql("""
select distinct(year(datadate)) from 'E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet'
""")

┌──────────────────┐
│ "year"(datadate) │
│      int64       │
├──────────────────┤
│             2022 │
│             2020 │
│             2021 │
│             2019 │
│             2017 │
│             2018 │
└──────────────────┘

In [1]:
import pandas as pd
from optbinning import OptimalBinning

df=pd.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')

optb_income = OptimalBinning(name='N_AVG_DEPOSIT_12M', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)
optb_income.fit(df['N_AVG_DEPOSIT_12M'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'N_AVG_DEPOSIT_12M':")
yay = optb_income.binning_table.build()

Binning table cho 'N_AVG_DEPOSIT_12M':


In [3]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 4911651.75)",7565147,0.6873,7343615,221532,0.029283,-0.242035,0.045267,0.005645
1,"[4911651.75, inf)",3441907,0.3127,3408804,33103,0.009618,0.891438,0.166721,0.020176
2,Special,0,0.0000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.0000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,11007054,1.0000,10752419,254635,0.023134,,0.211988,0.025821


In [9]:
import pandas as pd
from optbinning import OptimalBinning

# Đọc dữ liệu từ file parquet
# Vui lòng đảm bảo đường dẫn file là chính xác
file_path = 'E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet'
df = pd.read_parquet(file_path)

# --- THAY ĐỔI BẮT ĐẦU TỪ ĐÂY ---

# 1. Đảm bảo cột 'datadate' có định dạng datetime. 
#    Nếu cột của bạn đã ở định dạng datetime, dòng này vẫn an toàn để chạy.
df['datadate'] = pd.to_datetime(df['datadate'])

# 2. Lọc DataFrame để chỉ lấy các bản ghi có năm (year) của cột 'datadate' > 2017
#    Sử dụng .copy() để tránh cảnh báo SettingWithCopyWarning
df_filtered = df[df['datadate'].dt.year > 2017].copy()

# In ra số lượng bản ghi trước và sau khi lọc để kiểm tra
print(f"Số lượng bản ghi ban đầu: {len(df)}")
print(f"Số lượng bản ghi sau khi lọc (năm > 2017): {len(df_filtered)}")

# --- KẾT THÚC THAY ĐỔI ---


# Sử dụng DataFrame đã được lọc (df_filtered) cho các bước tiếp theo
optb_income = OptimalBinning(name='N_AVG_DEPOSIT_12M', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)

# Fit mô hình trên dữ liệu đã lọc
optb_income.fit(df_filtered['N_AVG_DEPOSIT_12M'].values, df_filtered['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("\nBinning table cho 'N_AVG_DEPOSIT_12M' (trên dữ liệu sau 2017):")
binning_table = optb_income.binning_table.build()

Số lượng bản ghi ban đầu: 11007054
Số lượng bản ghi sau khi lọc (năm > 2017): 9522184

Binning table cho 'N_AVG_DEPOSIT_12M' (trên dữ liệu sau 2017):


C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure

In [11]:
binning_table

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 4928137.50)",6251800,0.656551,6070286,181514,0.029034,-0.270864,0.054943,0.006847
1,"[4928137.50, inf)",3270384,0.343449,3239569,30815,0.009422,0.874502,0.177387,0.021493
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,9522184,1.000000,9309855,212329,0.022298,,0.232330,0.028340


In [13]:
import pandas as pd
from optbinning import OptimalBinning

optb_income = OptimalBinning(name='CBAL', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)
optb_income.fit(df_filtered['CBAL'].values, df_filtered['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'CBAL':")
yay = optb_income.binning_table.build()

Binning table cho 'CBAL':


C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\duclt6\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure

In [15]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1322317056.00)",8518791,0.894626,8337452,181339,0.021287,0.047453,0.001969,0.000246
1,"[1322317056.00, inf)",1003393,0.105374,972403,30990,0.030885,-0.334586,0.013887,0.001728
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,9522184,1.000000,9309855,212329,0.022298,,0.015856,0.001974


In [5]:
import pandas as pd
from optbinning import OptimalBinning

optb_income = OptimalBinning(name='CBAL', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2)
                             #min_prebin_size=0.1, 
                             #min_bin_size= 0.1)
optb_income.fit(df['CBAL'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'CBAL':")
yay = optb_income.binning_table.build()

Binning table cho 'CBAL':


In [6]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1511932224.00)",10091573,0.916828,9868021,223552,0.022152,0.044356,0.001766,0.000221
1,"[1511932224.00, inf)",915481,0.083172,884398,31083,0.033953,-0.394809,0.015720,0.001952
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,11007054,1.000000,10752419,254635,0.023134,,0.017487,0.002173


In [9]:
import pandas as pd
from optbinning import OptimalBinning

optb_income = OptimalBinning(name='N_AVG_DEPOSIT_12M', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2)
                             #min_prebin_size=0.1, 
                             #min_bin_size= 0.1)
optb_income.fit(df['N_AVG_DEPOSIT_12M'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'N_AVG_DEPOSIT_12M':")
yay = optb_income.binning_table.build()

Binning table cho 'N_AVG_DEPOSIT_12M':


In [10]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 2990646.75)",6793188,0.617167,6584563,208625,0.030711,-0.29111,0.060239,0.007503
1,"[2990646.75, inf)",4213866,0.382833,4167856,46010,0.010919,0.763243,0.157938,0.019277
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,11007054,1.000000,10752419,254635,0.023134,,0.218178,0.026780


In [13]:
import pandas as pd
from optbinning import OptimalBinning

optb_income = OptimalBinning(name='CBAL', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)
optb_income.fit(df['CBAL'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'CBAL':")
yay = optb_income.binning_table.build()

Binning table cho 'CBAL':


In [14]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1212699328.00)",9853906,0.895236,9636813,217093,0.022031,0.049965,0.002183,0.000273
1,"[1212699328.00, inf)",1153148,0.104764,1115606,37542,0.032556,-0.351362,0.015348,0.001909
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,11007054,1.000000,10752419,254635,0.023134,,0.017530,0.002181


In [17]:
import pandas as pd
import numpy as np


# 1. Định nghĩa các điều kiện cho từng phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] < 1511932224.00), # PK1
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] >= 1511932224.00),  # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] < 1511932224.00),  # PK3
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] >= 1511932224.00)   # PK4
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = ['PK1 (Số dư < 4tr, Dư nợ <900tr)', 'PK2 Số dư < 4tr, Dư nợ >= 900tr)', 'PK3 Số dư > 4tr, Dư nợ <900tr', 'PK4 Số dư > 4tr, Dư nợ >=900tr']

# 3. Tạo cột 'segment' mới dựa trên các điều kiện
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính tỷ lệ vỡ nợ (default rate) cho mỗi phân khúc
# Tỷ lệ vỡ nợ chính là giá trị trung bình của cột 'bad' (vì bad=1 là vỡ nợ, bad=0 là không)
default_rate_by_segment = df.groupby('segment')['BAD'].mean()

# 5. (Tùy chọn) Tính số lượng quan sát trong mỗi phân khúc
segment_counts = df['segment'].value_counts()

# 6. Kết hợp kết quả và hiển thị
results = pd.DataFrame({
    'Default Rate': default_rate_by_segment,
    'Count': segment_counts
})

print("Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:")
print(results)

# Bạn cũng có thể xem tỷ lệ dưới dạng phần trăm
results['Default Rate (%)'] = results['Default Rate'] * 100
print("\nTỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:")
print(results[['Default Rate (%)', 'Count']])

Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:
                                  Default Rate    Count
segment                                                
PK1 (Số dư < 4tr, Dư nợ <900tr)       0.027759  7177497
PK2 Số dư < 4tr, Dư nợ >= 900tr)      0.057508   387650
PK3 Số dư > 4tr, Dư nợ <900tr         0.008343  2914076
PK4 Số dư > 4tr, Dư nợ >=900tr        0.016653   527831

Tỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:
                                  Default Rate (%)    Count
segment                                                    
PK1 (Số dư < 4tr, Dư nợ <900tr)           2.775884  7177497
PK2 Số dư < 4tr, Dư nợ >= 900tr)          5.750806   387650
PK3 Số dư > 4tr, Dư nợ <900tr             0.834330  2914076
PK4 Số dư > 4tr, Dư nợ >=900tr            1.665306   527831


In [19]:
results

,Default Rate,Count,Default Rate (%)
segment,,,
"PK1 (Số dư < 4tr, Dư nợ <900tr)",0.027759,7177497,2.775884
"PK2 Số dư < 4tr, Dư nợ >= 900tr)",0.057508,387650,5.750806
"PK3 Số dư > 4tr, Dư nợ <900tr",0.008343,2914076,0.834330
"PK4 Số dư > 4tr, Dư nợ >=900tr",0.016653,527831,1.665306


In [33]:
import pandas as pd
import numpy as np


# 1. Định nghĩa các điều kiện cho từng phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 2990646.75) & (df['CBAL'] < 1511932224.00), # PK1
    (df['N_AVG_DEPOSIT_12M'] < 2990646.75) & (df['CBAL'] >= 1511932224.00),  # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 2990646.75) & (df['CBAL'] < 1511932224.00),  # PK3
    (df['N_AVG_DEPOSIT_12M'] >= 2990646.75) & (df['CBAL'] >= 1511932224.00)   # PK4
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = ['PK1 (Số dư < 3tr, Dư nợ <1t5)', 'PK2 Số dư < 3tr, Dư nợ >= 1t5)', 'PK3 Số dư > 3tr, Dư nợ <1t5', 'PK4 Số dư > 3tr, Dư nợ >=1t5']

# 3. Tạo cột 'segment' mới dựa trên các điều kiện
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính tỷ lệ vỡ nợ (default rate) cho mỗi phân khúc
# Tỷ lệ vỡ nợ chính là giá trị trung bình của cột 'bad' (vì bad=1 là vỡ nợ, bad=0 là không)
default_rate_by_segment = df.groupby('segment')['BAD'].mean()

# 5. (Tùy chọn) Tính số lượng quan sát trong mỗi phân khúc
segment_counts = df['segment'].value_counts()

# 6. Kết hợp kết quả và hiển thị
results = pd.DataFrame({
    'Default Rate': default_rate_by_segment,
    'Count': segment_counts
})

print("Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:")
print(results)

# Bạn cũng có thể xem tỷ lệ dưới dạng phần trăm
results['Default Rate (%)'] = results['Default Rate'] * 100
print("\nTỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:")
print(results[['Default Rate (%)', 'Count']])

Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:
                                Default Rate    Count
segment                                              
PK1 (Số dư < 3tr, Dư nợ <1t5)       0.029130  6470170
PK2 Số dư < 3tr, Dư nợ >= 1t5)      0.062380   323018
PK3 Số dư > 3tr, Dư nợ <1t5         0.009686  3621403
PK4 Số dư > 3tr, Dư nợ >=1t5        0.018453   592463

Tỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:
                                Default Rate (%)    Count
segment                                                  
PK1 (Số dư < 3tr, Dư nợ <1t5)           2.912984  6470170
PK2 Số dư < 3tr, Dư nợ >= 1t5)          6.238042   323018
PK3 Số dư > 3tr, Dư nợ <1t5             0.968603  3621403
PK4 Số dư > 3tr, Dư nợ >=1t5            1.845347   592463


In [34]:
results

,Default Rate,Count,Default Rate (%)
segment,,,
"PK1 (Số dư < 3tr, Dư nợ <1t5)",0.029130,6470170,2.912984
"PK2 Số dư < 3tr, Dư nợ >= 1t5)",0.062380,323018,6.238042
"PK3 Số dư > 3tr, Dư nợ <1t5",0.009686,3621403,0.968603
"PK4 Số dư > 3tr, Dư nợ >=1t5",0.018453,592463,1.845347


In [29]:
import pandas as pd
import numpy as np


# 1. Định nghĩa các điều kiện cho từng phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] < 1212699328.00), # PK1
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] >= 1212699328.00),  # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] < 1212699328.00),  # PK3
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] >= 1212699328.00)   # PK4
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = ['PK1 (Số dư < 4tr9, Dư nợ <1t2)', 'PK2 Số dư < 4tr9, Dư nợ >= 1t2)', 'PK3 Số dư > 4tr9, Dư nợ <1t2', 'PK4 Số dư > 4tr9, Dư nợ >=1t2']

# 3. Tạo cột 'segment' mới dựa trên các điều kiện
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính tỷ lệ vỡ nợ (default rate) cho mỗi phân khúc
# Tỷ lệ vỡ nợ chính là giá trị trung bình của cột 'bad' (vì bad=1 là vỡ nợ, bad=0 là không)
default_rate_by_segment = df.groupby('segment')['BAD'].mean()

# 5. (Tùy chọn) Tính số lượng quan sát trong mỗi phân khúc
segment_counts = df['segment'].value_counts()

# 6. Kết hợp kết quả và hiển thị
results = pd.DataFrame({
    'Default Rate': default_rate_by_segment,
    'Count': segment_counts
})

print("Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:")
print(results)

# Bạn cũng có thể xem tỷ lệ dưới dạng phần trăm
results['Default Rate (%)'] = results['Default Rate'] * 100
print("\nTỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:")
print(results[['Default Rate (%)', 'Count']])

Tỷ lệ vỡ nợ và số lượng theo từng phân khúc:
                                 Default Rate    Count
segment                                               
PK1 (Số dư < 4tr9, Dư nợ <1t2)       0.027525  7050829
PK2 Số dư < 4tr9, Dư nợ >= 1t2)      0.053385   514318
PK3 Số dư > 4tr9, Dư nợ <1t2         0.008212  2803077
PK4 Số dư > 4tr9, Dư nợ >=1t2        0.015787   638830

Tỷ lệ vỡ nợ (%) và số lượng theo từng phân khúc:
                                 Default Rate (%)    Count
segment                                                   
PK1 (Số dư < 4tr9, Dư nợ <1t2)           2.752513  7050829
PK2 Số dư < 4tr9, Dư nợ >= 1t2)          5.338526   514318
PK3 Số dư > 4tr9, Dư nợ <1t2             0.821169  2803077
PK4 Số dư > 4tr9, Dư nợ >=1t2            1.578667   638830


In [1]:
import pandas as pd
import numpy as np

df=pd.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')
# Giả sử bạn đã có DataFrame df với các cột:
# 'N_AVG_DEPOSIT_12M' (số dư trung bình 12 tháng),
# 'CBAL' (dư nợ), và 'BAD' (1 = vỡ nợ, 0 = không vỡ nợ)

# 1. Định nghĩa các điều kiện cho từng phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] < 1212699328.00),  # PK1
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] >= 1212699328.00), # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] < 1212699328.00), # PK3
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75) & (df['CBAL'] >= 1212699328.00) # PK4
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = [
    'PK1 (Số dư <4.9tr, Dư nợ <1.2t)', 
    'PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)', 
    'PK3 (Số dư ≥4.9tr, Dư nợ <1.2t)', 
    'PK4 (Số dư ≥4.9tr, Dư nợ ≥1.2t)'
]

# 3. Tạo cột 'segment'
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính thống kê: tổng số, số bad, số good, tỷ lệ vỡ nợ (%)
stats = (
    df.groupby('segment')['BAD']
      .agg(
          Count='size',           # tổng số quan sát
          Bad='sum',              # số vỡ nợ
          Default_Rate='mean'     # tỷ lệ vỡ nợ (0–1)
      )
      .reset_index()
)

# 5. Tính số good và tỷ lệ vỡ nợ (%)
stats['Good'] = stats['Count'] - stats['Bad']
stats['Default Rate (%)'] = stats['Default_Rate'] * 100

# 6. Sắp xếp lại cột cho dễ nhìn
stats = stats[['segment', 'Count', 'Good', 'Bad', 'Default Rate (%)']]

# 7. In kết quả
print("Thống kê theo phân khúc:")
print(stats.to_string(index=False))


Thống kê theo phân khúc:
                        segment   Count    Good    Bad  Default Rate (%)
PK1 (Số dư <4.9tr, Dư nợ <1.2t) 7050829 6856754 194075          2.752513
PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)  514318  486861  27457          5.338526
PK3 (Số dư ≥4.9tr, Dư nợ <1.2t) 2803077 2780059  23018          0.821169
PK4 (Số dư ≥4.9tr, Dư nợ ≥1.2t)  638830  628745  10085          1.578667


In [3]:
import pandas as pd
import numpy as np

# Giả sử bạn đã có DataFrame df với các cột:
# 'N_AVG_DEPOSIT_12M' (số dư trung bình 12 tháng),
# 'CBAL' (dư nợ), và 'BAD' (1 = vỡ nợ, 0 = không vỡ nợ)

# 1. Định nghĩa các điều kiện cho từng phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] < 1212699328.00),  # PK1
    (df['N_AVG_DEPOSIT_12M'] < 4911651.75) & (df['CBAL'] >= 1212699328.00), # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 4911651.75)
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = [
    'PK1 (Số dư <4.9tr, Dư nợ <1.2t)', 
    'PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)', 
    'PK3 (Số dư ≥4.9tr)'

]

# 3. Tạo cột 'segment'
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính thống kê: tổng số, số bad, số good, tỷ lệ vỡ nợ (%)
stats = (
    df.groupby('segment')['BAD']
      .agg(
          Count='size',           # tổng số quan sát
          Bad='sum',              # số vỡ nợ
          Default_Rate='mean'     # tỷ lệ vỡ nợ (0–1)
      )
      .reset_index()
)

# 5. Tính số good và tỷ lệ vỡ nợ (%)
stats['Good'] = stats['Count'] - stats['Bad']
stats['Default Rate (%)'] = stats['Default_Rate'] * 100

# 6. Sắp xếp lại cột cho dễ nhìn
stats = stats[['segment', 'Count', 'Good', 'Bad', 'Default Rate (%)']]

# 7. In kết quả
print("Thống kê theo phân khúc:")
print(stats.to_string(index=False))


Thống kê theo phân khúc:
                        segment   Count    Good    Bad  Default Rate (%)
PK1 (Số dư <4.9tr, Dư nợ <1.2t) 7050829 6856754 194075          2.752513
PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)  514318  486861  27457          5.338526
             PK3 (Số dư ≥4.9tr) 3441907 3408804  33103          0.961763


In [7]:
stats

,segment,Count,Good,Bad,Default Rate (%)
0,"PK1 (Số dư <4.9tr, Dư nợ <1.2t)",7050829,6856754,194075,2.752513
1,"PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)",514318,486861,27457,5.338526
2,PK3 (Số dư ≥4.9tr),3441907,3408804,33103,0.961763


In [13]:
from duckdb import sql
sql("""
select max(CBAL) from 'E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/SEGMENT_1/BCARD_DATA2_MIN_1_20252304.parquet'
""")

┌─────────────────┐
│    max(CBAL)    │
│     double      │
├─────────────────┤
│ 705437589.36818 │
└─────────────────┘

In [23]:
import pandas as pd
import numpy as np


conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 5000000) & (df['CBAL'] < 1200000000),  # PK1
    (df['N_AVG_DEPOSIT_12M'] < 5000000) & (df['CBAL'] >= 1200000000), # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 5000000)
]

# 2. Định nghĩa tên cho từng phân khúc
segment_names = [
    'PK1 (Số dư <5tr, Dư nợ <1.2t)', 
    'PK2 (Số dư <5tr, Dư nợ ≥1.2t)', 
    'PK3 (Số dư ≥5tr)'

]

# 3. Tạo cột 'segment'
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Tính thống kê: tổng số, số bad, số good, tỷ lệ vỡ nợ (%)
stats = (
    df.groupby('segment')['BAD']
      .agg(
          Count='size',           # tổng số quan sát
          Bad='sum',              # số vỡ nợ
          Default_Rate='mean'     # tỷ lệ vỡ nợ (0–1)
      )
      .reset_index()
)
total_count = stats['Count'].sum()
# 5. Tính số good và tỷ lệ vỡ nợ (%)
stats['Good'] = stats['Count'] - stats['Bad']
stats['Default Rate (%)'] = stats['Default_Rate'] * 100
stats['Count%'] = (stats['Count'] / total_count) * 100
# 6. Sắp xếp lại cột cho dễ nhìn
stats = stats[['segment', 'Count%', 'Count', 'Good', 'Bad', 'Default Rate (%)']]

# 7. In kết quả
print("Thống kê theo phân khúc:")
print(stats.to_string(index=False))


Thống kê theo phân khúc:
                        segment    Count%   Count    Good    Bad  Default Rate (%)
PK1 (Số dư <4.9tr, Dư nợ <1.2t) 64.153715 7061434 6867396 194038          2.747855
PK2 (Số dư <4.9tr, Dư nợ ≥1.2t)  4.816093  530110  502262  27848          5.253249
             PK3 (Số dư ≥4.9tr) 31.030192 3415510 3382761  32749          0.958832


In [5]:
import pandas as pd
import numpy as np

# Giả sử df đã có các cột: 'N_AVG_DEPOSIT_12M', 'CBAL', 'BAD'
#df=pd.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')
# 1. Định nghĩa điều kiện phân khúc
conditions = [
    (df['N_AVG_DEPOSIT_12M'] < 5_000_000) & (df['CBAL'] < 1_200_000_000),   # PK1
    (df['N_AVG_DEPOSIT_12M'] < 5_000_000) & (df['CBAL'] >= 1_200_000_000),  # PK2
    (df['N_AVG_DEPOSIT_12M'] >= 5_000_000)                                  # PK3
]

# 2. Tên các phân khúc
segment_names = [
    'Deposit ∈ (-∞, 5M), CBAL ∈ (-∞, 1.2B)', 
    'Deposit ∈ (-∞, 5M), CBAL ∈ [1.2B, ∞)', 
    'Deposit ∈ [5M, ∞)'
]

# 3. Gán phân khúc
df['segment'] = np.select(conditions, segment_names, default='Unknown')

# 4. Thống kê cơ bản theo segment
stats = (
    df.groupby('segment')['BAD']
      .agg(
          Count='size',             # Tổng số quan sát
          Bad='sum',                # Số vỡ nợ
          Default_Rate='mean'       # Tỷ lệ vỡ nợ (0–1)
      )
      .reset_index()
)

# 5. Tính thêm Good, tỷ lệ % và Default Rate %
total_count = stats['Count'].sum()
stats['Good'] = stats['Count'] - stats['Bad']
stats['Default Rate (%)'] = stats['Default_Rate'] * 100
stats['Count%'] = (stats['Count'] / total_count) * 100

# 6. Tính WoE và IV
total_good = df['BAD'].value_counts().get(0, 0)
total_bad = df['BAD'].value_counts().get(1, 0)

def calculate_woe_iv(row):
    dist_good = row['Good'] / total_good if total_good > 0 else 0
    dist_bad = row['Bad'] / total_bad if total_bad > 0 else 0
    woe = np.log(dist_good / dist_bad) if dist_good > 0 and dist_bad > 0 else 0
    iv = (dist_good - dist_bad) * woe
    return pd.Series({'WoE': woe, 'IV': iv})

woe_iv = stats.apply(calculate_woe_iv, axis=1)
stats = pd.concat([stats, woe_iv], axis=1)

# 7. Chọn và sắp xếp lại cột
stats = stats[['segment', 'Count%', 'Count', 'Good', 'Bad', 'Default Rate (%)', 'WoE', 'IV']]

# 8. In kết quả
print("Thống kê theo phân khúc có WoE & IV:")
print(stats.to_string(index=False))

# 9. Tổng IV toàn biến 'segment'
total_iv = stats['IV'].sum()
print(f"\nTổng Information Value (IV) của biến 'segment': {total_iv:.4f}")


Thống kê theo phân khúc có WoE & IV:
                              segment    Count%   Count    Good    Bad  Default Rate (%)       WoE       IV
Deposit ∈ (-∞, 5M), CBAL ∈ (-∞, 1.2B) 64.153715 7061434 6867396 194038          2.747855 -0.176569 0.021778
 Deposit ∈ (-∞, 5M), CBAL ∈ [1.2B, ∞)  4.816093  530110  502262  27848          5.253249 -0.850694 0.053298
                    Deposit ∈ [5M, ∞) 31.030192 3415510 3382761  32749          0.958832  0.894520 0.166375

Tổng Information Value (IV) của biến 'segment': 0.2415


In [9]:
stats.to_excel('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/Segment_result_final.xlsx',index=False)

In [5]:
7050829+514318+3441907

11007054

In [30]:
results

,Default Rate,Count,Default Rate (%)
segment,,,
"PK1 (Số dư < 4tr9, Dư nợ <1t2)",0.027525,7050829,2.752513
"PK2 Số dư < 4tr9, Dư nợ >= 1t2)",0.053385,514318,5.338526
"PK3 Số dư > 4tr9, Dư nợ <1t2",0.008212,2803077,0.821169
"PK4 Số dư > 4tr9, Dư nợ >=1t2",0.015787,638830,1.578667


In [43]:
0.008212*2

0.016424

In [11]:
# Cắt dữ liệu theo điều kiện phân nhóm
df=pd.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')
# df_cut1: N_AVG_DEPOSIT_12M < 4,911,651.75 và CBAL < 1,212,699,328.00
df_cut1 = df[
    (df['N_AVG_DEPOSIT_12M'] < 5000000) &
    (df['CBAL'] < 1200000000)
]

# df_cut2: N_AVG_DEPOSIT_12M < 4,911,651.75 và CBAL >= 1,212,699,328.00
df_cut2 = df[
    (df['N_AVG_DEPOSIT_12M'] < 5000000) &
    (df['CBAL'] >= 1200000000)
]

# df_cut3: N_AVG_DEPOSIT_12M > 4,911,651.75
df_cut3 = df[
    (df['N_AVG_DEPOSIT_12M'] > 5000000)
]


In [28]:


df_cut1.to_parquet('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/SEGMENT_1_NEW/segment_1_v2.parquet')
df_cut2.to_parquet('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/SEGMENT_2_NEW/segment_2_v2.parquet')
df_cut3.to_parquet('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/SEGMENT_3_NEW/segment_3_v2.parquet')

In [15]:
from duckdb import sql
sql("""
select count(*) from df_cut1
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      7061434 │
└──────────────┘

In [17]:
sql("""
select count(*) from df_cut2
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       530110 │
└──────────────┘

In [19]:
sql("""
select count(*) from df_cut3
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      3415440 │
└──────────────┘

In [21]:
sql("""
select count(*) from df
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     11007054 │
└──────────────┘

In [33]:
7061434+530110+3415440

11006984

In [25]:
df_cut3_new = df[
    (df['N_AVG_DEPOSIT_12M'] >= 5000000)
]

In [27]:
sql("""
select count(*) from df_cut3_new
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      3415510 │
└──────────────┘

In [35]:
3415510 - 3415440

70

In [37]:
df_cut3_new

,datadate,SOCIF,CBALORG,CBALORG_AVG,CBALORG_MAX,CBALORG_MIN,CBAL,CBAL_AVG,CBAL_MAX,CBAL_MIN,...,MAX_DEBTGRADE_12M,NHOMNOCUOI_PREV,NHOMNOCUOI_CUR,NHOMNOCIC,NHOMNOCUOI,XULYNO,BAD,C_GIOITINH,final_CST_MKT_SEG,PDKH
1,2021-03-31,9466306.0,1.300000e+09,1.300000e+09,1.300000e+09,1.300000e+09,1.301870e+09,1.301870e+09,1.301870e+09,1.301870e+09,...,1.0,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 2,KHPT
4,2019-02-28,7239730.0,5.143300e+08,2.571650e+08,3.593300e+08,1.550000e+08,5.176336e+08,2.588168e+08,3.615938e+08,1.560398e+08,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT
7,2018-06-30,1926120.0,1.000000e+08,1.000000e+08,1.000000e+08,1.000000e+08,1.001890e+08,1.001890e+08,1.001890e+08,1.001890e+08,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT
12,2018-07-31,4300957.0,1.166667e+08,1.166667e+08,1.166667e+08,1.166667e+08,1.169276e+08,1.169276e+08,1.169276e+08,1.169276e+08,...,1.0,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 2,KHPT
13,2020-08-31,10269194.0,8.000000e+09,8.000000e+09,8.000000e+09,8.000000e+09,8.011412e+09,8.011412e+09,8.011412e+09,8.011412e+09,...,1.0,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 1,KHPT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11007037,2019-04-30,3202417.0,5.680000e+07,5.680000e+07,5.680000e+07,5.680000e+07,5.691833e+07,5.691833e+07,5.691833e+07,5.691833e+07,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Vàng,KHCC
11007041,2020-08-31,9220753.0,7.332000e+08,7.332000e+08,7.332000e+08,7.332000e+08,7.345358e+08,7.345358e+08,7.345358e+08,7.345358e+08,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 2,KHPT
11007047,2021-04-30,2616126.0,3.000000e+07,3.000000e+07,3.000000e+07,3.000000e+07,3.070861e+07,3.070861e+07,3.070861e+07,3.070861e+07,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 3,KHPT
11007048,2022-11-30,1247522.0,6.000000e+08,6.000000e+08,6.000000e+08,6.000000e+08,6.044110e+08,6.044110e+08,6.044110e+08,6.044110e+08,...,1.0,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông,KHPT


In [39]:
df_cut3_new.to_parquet('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/FINAL_MODEL/SEGMENT_3_NEW/segment_3_v3.parquet')

In [53]:
df_cut1

,datadate,SOCIF,CBALORG,CBALORG_AVG,CBALORG_MAX,CBALORG_MIN,CBAL,CBAL_AVG,CBAL_MAX,CBAL_MIN,...,NHOMNOCUOI_PREV,NHOMNOCUOI_CUR,NHOMNOCIC,NHOMNOCUOI,XULYNO,BAD,C_GIOITINH,final_CST_MKT_SEG,PDKH,segment
0,2021-02-28,11206177.0,4.966770e+08,4.966770e+08,4.966770e+08,4.966770e+08,4.971056e+08,4.971056e+08,4.971056e+08,4.971056e+08,...,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
2,2021-11-30,9773841.0,2.786000e+08,2.786000e+08,2.786000e+08,2.786000e+08,2.798835e+08,2.798835e+08,2.798835e+08,2.798835e+08,...,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
3,2022-03-31,14912774.0,1.531000e+08,1.531000e+08,1.531000e+08,1.531000e+08,1.543374e+08,1.543374e+08,1.543374e+08,1.543374e+08,...,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
5,2019-02-28,9997855.0,5.528000e+08,5.528000e+08,5.528000e+08,5.528000e+08,5.532952e+08,5.532952e+08,5.532952e+08,5.532952e+08,...,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
6,2020-06-30,10655949.0,1.895930e+08,1.895930e+08,1.895930e+08,1.895930e+08,1.899146e+08,1.899146e+08,1.899146e+08,1.899146e+08,...,1.0,1.0,1.0,1.0,0,1,1.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11007046,2018-07-31,4116856.0,2.240000e+07,2.240000e+07,2.240000e+07,2.240000e+07,2.256290e+07,2.256290e+07,2.256290e+07,2.256290e+07,...,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
11007049,2021-06-30,11484287.0,1.000000e+09,1.000000e+09,1.000000e+09,1.000000e+09,1.007479e+09,1.007479e+09,1.007479e+09,1.007479e+09,...,1.0,1.0,1.0,1.0,0,0,1.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
11007050,2017-01-31,4363671.0,1.125000e+07,1.125000e+07,1.125000e+07,1.125000e+07,1.114844e+07,1.114844e+07,1.114844e+07,1.114844e+07,...,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"
11007051,2017-09-30,5444107.0,4.472800e+07,4.472800e+07,4.472800e+07,4.472800e+07,4.482118e+07,4.482118e+07,4.482118e+07,4.482118e+07,...,1.0,1.0,1.0,1.0,0,0,2.0,Phổ thông 1,KHPT,"PK1 (Số dư < 3tr, Dư nợ <1t5)"


In [15]:
X_2 = file_pl.select(['CBALORG', 'CBALORG_AVG', 'CBAL', 'CBAL_AVG', 'N_AVG_CBAL_12M', 'N_AVG_CBAL_9M', 'N_AVG_CBAL_6M', 'N_AVG_CBAL_3M'])

for i in X_2.columns:
    X_2 = X_2.with_columns(
        pl.when(pl.col(i) > 0)
        .then(pl.col(i).log()) 
        .otherwise(np.log(1e-10))  
        .alias(f"{i}_log")
    )

# DecisionTreeClassifier 
model_2 = DecisionTreeClassifier(
        criterion = 'gini',
        splitter='best',
        max_depth=2,
        random_state=123,
        class_weight='balanced'
    )

X_2_np = X_2.to_numpy()

model_2.fit(X_2_np, y_np)

DecisionTreeClassifier(class_weight='balanced', max_depth=2, random_state=123)

In [16]:
# Lấy độ quan trọng
importances_1 = model_2.feature_importances_
feature_names_1 = X_2.columns  # Nếu X là DataFrame

# Tạo bảng
feat_imp_1 = pd.Series(importances_1, index=feature_names_1).sort_values(ascending=False)

# In ra top feature quan trọng nhất
print("🎯 Feature quan trọng:")
feat_imp_1

🎯 Feature quan trọng:


CBAL                  0.94979
CBAL_log              0.05021
CBALORG               0.00000
CBALORG_AVG           0.00000
CBAL_AVG              0.00000
N_AVG_CBAL_12M        0.00000
N_AVG_CBAL_9M         0.00000
N_AVG_CBAL_6M         0.00000
N_AVG_CBAL_3M         0.00000
CBALORG_log           0.00000
CBALORG_AVG_log       0.00000
CBAL_AVG_log          0.00000
N_AVG_CBAL_12M_log    0.00000
N_AVG_CBAL_9M_log     0.00000
N_AVG_CBAL_6M_log     0.00000
N_AVG_CBAL_3M_log     0.00000
dtype: float64

Chọn 2 biến N_AVG_DEPOSIT_3M và CBAL để segment

In [24]:
X_f = file_pl.select(['N_AVG_DEPOSIT_3M','CBAL'])

for i in X_f.columns:
    X_f = X_f.with_columns(
        pl.when(pl.col(i) > 0)
        .then(pl.col(i).log()) 
        .otherwise(np.log(1e-10))  
        .alias(f"{i}_log")
    )

model_f = DecisionTreeClassifier(
        criterion = 'gini',
        splitter='best',
        max_depth=2,
        random_state=123,
        class_weight='balanced'
    )

model_f.fit(X_f.select(['N_AVG_DEPOSIT_3M','CBAL']).to_numpy(),y)

#plt.figure(figsize=(20,10))

#plot_tree(model_f, 
 #         feature_names= ['N_AVG_DEPOSIT_3M','CBAL'], 
  #        class_names=[str(c) for c in model_f.classes_],
  #        filled=True,
   #       rounded=True)
          
#plt.show()

DecisionTreeClassifier(class_weight='balanced', max_depth=2, random_state=123)

In [28]:
# Lấy độ quan trọng
importances_2 = model_f.feature_importances_
feature_names_2 = ['N_AVG_DEPOSIT_3M','N_AVG_DEPOSIT_3M_log']  # Nếu X là DataFrame

# Tạo bảng
feat_imp_2 = pd.Series(importances_2, index=feature_names_2)

# In ra top feature quan trọng nhất
print("🎯 Feature quan trọng:")
feat_imp_2

🎯 Feature quan trọng:


N_AVG_DEPOSIT_3M        1.0
N_AVG_DEPOSIT_3M_log    0.0
dtype: float64

### 7. Segment

In [24]:
from optbinning import OptimalBinning

hehe= pd.DataFrame()
haha= pd.DataFrame()

for i in ['N_AVG_DEPOSIT_3M']:
    optb = OptimalBinning(name=i, 
                          dtype='numerical', 
                          prebinning_method='cart', 
                          solver='cp', 
                          divergence='iv', 
                          max_n_prebins= 20, 
                          min_prebin_size=0.1, 
                          min_n_bins=2, 
                          max_n_bins=2, 
                          min_bin_size= 0.1, 
                          max_bin_size= 0.9, 
                        verbose=False
                        )
    optb.fit(file_pl.select(i).to_numpy().ravel(), 
            y.to_numpy().ravel())
    table = pd.DataFrame(optb.binning_table.build())
    table['Feature'] = i
    haha = pd.concat([table.tail(1),haha], axis= 0)
    table = table[~table['Bin'].isin(['Special','Missing',''])]
    hehe = pd.concat([table, hehe], axis= 0)
    

In [38]:
from duckdb import sql
sql("""
select count(*) from 'E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/BCARD_DATA_FIXED_NONFILL.parquet'
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     13443454 │
└──────────────┘

In [25]:
heh = hehe.merge(haha[['Feature', 'IV']], on='Feature', how= 'left')
#heh.to_excel("E:/Bcard/Result/logistic result/Segment results.xlsx")

In [26]:
heh[['Feature','Feature','Bin', 'Event rate', 'Count', 'Count (%)', 'Non-event', 'Event',  'IV_y']].sort_values('IV_y', ascending= False)

,Feature,Feature,Bin,Event rate,Count,Count (%),Non-event,Event,IV_y
0,N_AVG_DEPOSIT_3M,N_AVG_DEPOSIT_3M,"(-inf, 3595988.50)",0.030322,8147854,0.606083,7900792,247062,0.326848
1,N_AVG_DEPOSIT_3M,N_AVG_DEPOSIT_3M,"[3595988.50, inf)",0.008046,4950954,0.368280,4911119,39835,0.326848


In [9]:
import pandas as pd
from optbinning import OptimalBinning

df=pd.read_parquet('E:/Duc/FINAL_CODE_BCARD_FULL_SEGMENT/BCARD_DATA_FIXED_NONFILL.parquet')

optb_income = OptimalBinning(name='N_AVG_DEPOSIT_3M', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)
optb_income.fit(df['N_AVG_DEPOSIT_3M'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'N_AVG_DEPOSIT_3M':")
yay = optb_income.binning_table.build()

Binning table cho 'N_AVG_DEPOSIT_3M':


In [5]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1835316.19)",6992562,0.520146,6764361,228201,0.032635,-0.393917,0.097867,0.012155
1,"[1835316.19, inf)",6106246,0.454217,6047550,58696,0.009612,0.851924,0.224751,0.027274
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,344646,0.025637,332485,12161,0.035285,-0.474753,0.007297,0.000904
Totals,,13443454,1.000000,13144396,299058,0.022246,,0.329915,0.040333


In [11]:
yay

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 3595988.50)",8147854,0.606083,7900792,247062,0.030322,-0.318034,0.071576,0.008909
1,"[3595988.50, inf)",4950954,0.368280,4911119,39835,0.008046,1.031398,0.247976,0.029692
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,344646,0.025637,332485,12161,0.035285,-0.474753,0.007297,0.000904
Totals,,13443454,1.000000,13144396,299058,0.022246,,0.326848,0.039505


In [13]:
import pandas as pd
from optbinning import OptimalBinning

df=pd.read_parquet('E:/Duc/DA_KĐ_Xay_lai_mo_hinh_KHCN/DATA/B_CARD_TO_SENT_FULL/DELOITTE_TEMP_DB/BCARD_DATA2_MIN_FIXED_SEG.parquet')

optb_income = OptimalBinning(name='N_AVG_DEPOSIT_3M', 
                             dtype="numerical", 
                             solver="cp", 
                             max_n_bins=2,
                             min_prebin_size=0.1, 
                             min_bin_size= 0.1)
optb_income.fit(df['N_AVG_DEPOSIT_3M'].values, df['BAD'].values)

# Hiển thị bảng phân bin cho 'N_AVG_DEPOSIT_12M'
print("Binning table cho 'N_AVG_DEPOSIT_3M':")
ye = optb_income.binning_table.build()

Binning table cho 'N_AVG_DEPOSIT_3M':


In [15]:
ye

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1459681.50)",6193893,0.56272,5992258,201635,0.032554,-0.35129,0.082400,0.010247
1,"[1459681.50, inf)",4813161,0.43728,4760161,53000,0.011011,0.75469,0.177024,0.021617
2,Special,0,0.00000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.00000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,11007054,1.00000,10752419,254635,0.023134,,0.259424,0.031865


In [23]:
haha

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS,Feature
Totals,,13443454,1.0,13144396,299058,0.022246,,0.329915,0.040333,N_AVG_DEPOSIT_3M


### 8. Tạo sample

In [ ]:
import polars as pl
population = pl.read_parquet('F:\DELOITTE_TEMP_DB\BCARD_DATA2_MIN_FIXED.parquet')

In [ ]:
sample_1 = population.filter(pl.col("N_AVG_DEPOSIT_3M") < 1835316.19)
sample_2 = population.filter(pl.col("N_AVG_DEPOSIT_3M") >= 1835316.19)


In [ ]:
sample_1.write_parquet("F:\DELOITTE_TEMP_DB\BCARD_DATA3_MIN_FIXED_17APR_LOWER_DPS_3M_2PK.parquet")
sample_2.write_parquet("F:\DELOITTE_TEMP_DB\BCARD_DATA3_MIN_FIXED_17APR_HIGHER_DPS_3M_2PK.parquet")